In [1]:
from pyecharts import options as opts
from pyecharts.charts import Kline, Line, Bar, Grid, Scatter
from pyecharts.commons.utils import JsCode
from pyecharts.globals import ThemeType


from backtest import *
from technical_analysis import *
from utils import *

import shioaji as sj

In [2]:
# draw line func
def set_global_opts(
    charts,
    title=None,
    filter_mode='filter',
    show_xaxis=True,
    legend_opts={'is_show': False},
    tooltip_position=None,
    always_show_tooltip_content=False,
    toolbox_position=('95%', None),
    y_split_number=5,
    axis_index=None,
    grid_index=None,
    datazoom_position='95%',
    range_=(90, 100),
):
    charts.set_global_opts(
        title_opts=opts.TitleOpts(title=title),
        legend_opts=opts.LegendOpts(
            **legend_opts
        ),
        toolbox_opts=opts.ToolboxOpts(
            orient='vertical',
            pos_left=toolbox_position[0],
            pos_top=toolbox_position[1],
            is_show=True,
            feature=opts.ToolBoxFeatureOpts(
                save_as_image=opts.ToolBoxFeatureSaveAsImageOpts(
                    name=datetime.datetime.now(),
                    title='Download plot as png',
                    type_='png',
                    background_color='white',
                    pixel_ratio=1,
                ),
                restore=opts.ToolBoxFeatureRestoreOpts(
                    title='Restore',
                ),
                data_view=None,
                data_zoom=opts.ToolBoxFeatureDataZoomOpts(
                    zoom_title='Zoom',
                    back_title='Unzoom',
                    xaxis_index=None,
                    yaxis_index=None,
                ),  # 要另外設定DataZoomOpts
                magic_type=None,
                brush=opts.ToolBoxFeatureBrushOpts(
                    rect_title='Box Select',
                    polygon_title='Polygon Select',
                    line_x_title='H Select',
                    line_y_title='V Select',
                    keep_title='Keep Select',
                    clear_title='Clear Select',
                ),  # 要另外設定BrushOpts
            )
        ),
        datazoom_opts=[
            opts.DataZoomOpts(
                is_show=False,
                type_='inside',
                xaxis_index=axis_index,
                range_start=range_[0],
                range_end=range_[1],
                filter_mode=filter_mode,
            ),
            opts.DataZoomOpts(
                is_show=False,
                yaxis_index=None,
                type_='slider',
                orient='vertical',
                range_start=0,
                range_end=100,
                is_zoom_lock=True,
                filter_mode=filter_mode,
            ),
            opts.DataZoomOpts(
                is_show=True,
                xaxis_index=axis_index,
                type_='slider',
                pos_top=datazoom_position,
                range_start=range_[0],
                range_end=range_[1],
                filter_mode=filter_mode,
            ),
        ],
        xaxis_opts=opts.AxisOpts(
            is_show=show_xaxis,
            type_="category",
            is_scale=True,
            grid_index=grid_index,
            min_="dataMin",
            max_="dataMax",
        ),
        yaxis_opts=opts.AxisOpts(
            is_scale=True,
            grid_index=grid_index,
            split_number=y_split_number,
            splitarea_opts=opts.SplitAreaOpts(
                is_show=True, areastyle_opts=opts.AreaStyleOpts(opacity=1)
            ),
            splitline_opts=opts.SplitLineOpts(
                is_show=True, linestyle_opts=opts.LineStyleOpts(
                    width=1, color='#7B7B7B', opacity=0.5
                )
            ),
        ),
        tooltip_opts=opts.TooltipOpts(
            is_always_show_content=always_show_tooltip_content,
            position=tooltip_position,
            trigger_on='mousemove|click',
            trigger='axis',
            axis_pointer_type='line',
            background_color='rgba(245, 245, 245, 0.75)',
            border_width=1,
            border_color='#ccc',
            textstyle_opts=opts.TextStyleOpts(color='#000'),
        ),
        axispointer_opts=opts.AxisPointerOpts(
            is_show=True,
            link=[{'xAxisIndex': 'all'}],  # sharex
            label=opts.LabelOpts(background_color='#777'),
        ),
        brush_opts=opts.BrushOpts(  # 改icon標題要去ToolBoxFeatureBrushOpts改
            tool_box=['rect', 'polygon', 'lineX', 'lineY', 'keep', 'clear'],
            x_axis_index='all',
            brush_link='all',
            out_of_brush={'colorAlpha': 0.1},
            brush_style={
                'color': 'rgba(255, 232, 219, 0.3)',
                'borderColor': 'rgba(133, 44, 0, 0.3)'
            }
        ),
    )


def draw_klines(
    df,
    ma_keys=['sma5', 'sma10', 'sma20', 'sma60'],
    vwap=True,
    bbands=True
):
    xdata = df.index.tolist()
    ydata = np.round(df[['open', 'close', 'low', 'high']].values, 2).tolist()

    # 加資料
    kline = (
        Kline(
            init_opts=opts.InitOpts(
                theme=ThemeType.CHALK,
                bg_color='#23262F',
            )
        )
        .add_xaxis(xdata)
        .add_yaxis(
            series_name='Kline',
            y_axis=ydata,
            itemstyle_opts=opts.ItemStyleOpts(
                color='#FF0000',  # 陽
                color0='#28FF28',  # 陰
                border_color='#FF0000',
                border_color0='#28FF28',
            ),
            markpoint_opts=opts.MarkPointOpts(
                data=[
                    opts.MarkPointItem(value_dim='highest',
                                       type_="max", symbol_size=1),
                    opts.MarkPointItem(value_dim='lowest',
                                       type_="min", symbol_size=1),
                ]
            ),
        )
    )

    line = Line().add_xaxis(xdata)

    if ma_keys is not None:
        mas = np.round(df[ma_keys].values, 2).T.tolist()
        for i, key, in enumerate(ma_keys):
            line.add_yaxis(
                key,
                mas[i],
                is_smooth=True,
                symbol_size=0.1,  # 為了能顯示tooltip又不想顯示symbol
                is_hover_animation=False,
                linestyle_opts=opts.LineStyleOpts(opacity=0.75),
                label_opts=opts.LabelOpts(is_show=False),
            )
    if vwap:
        vwap = np.round(df['vwap'].values, 2).T.tolist()
        line.add_yaxis(
            'vwap',
            vwap,
            is_smooth=True,
            is_selected=False,
            symbol_size=0.1,  # 為了能顯示tooltip又不想顯示symbol
            is_hover_animation=False,
            linestyle_opts=opts.LineStyleOpts(width=2, color='#FFFF37'),
            label_opts=opts.LabelOpts(is_show=False),
        )
    if bbands:
        keys = ['bbands_up', 'bbands_mid', 'bbands_low']
        colors = ['#FFD306', '#00FFFF', '#FFD306']
        bbands_datas = np.round(df[keys].values).T.tolist()
        for i, (key, color) in enumerate(zip(keys, colors)):
            line.add_yaxis(
                key,
                bbands_datas[i],
                is_smooth=True,
                is_selected=False,
                symbol_size=0.1,
                is_hover_animation=False,
                linestyle_opts=opts.LineStyleOpts(width=1.5, color=color),
                label_opts=opts.LabelOpts(is_show=False),
            )
    kline.overlap(line)
    return kline


def draw_volume(
    df,
    ma_keys=['sma5_v', 'sma10_v'],
    grid_index=None,
    category_gap='20%',
):
    xdata = df.index.tolist()
    ydata = df.volume.values.tolist()

    color_func = """
        function(params) {
            var colorList;
            if (sign[params.dataIndex] >= 0) {
                colorList = '#FF0000';
            } else {
                colorList = '#28FF28';
            }
            return colorList;
        }
    """

    bar = (
        Bar(
            init_opts=opts.InitOpts(
                theme=ThemeType.CHALK,
                bg_color='#23262F',
            )
        )
        .add_xaxis(xdata)
        .add_yaxis(
            series_name='Volume',
            y_axis=ydata,
            category_gap=category_gap,
            xaxis_index=grid_index,
            yaxis_index=grid_index,
            label_opts=opts.LabelOpts(is_show=False),
            itemstyle_opts=opts.ItemStyleOpts(
                opacity=0.95, color=JsCode(color_func))
        )
    )

    line = Line().add_xaxis(xdata)
    if ma_keys is not None:
        mas = np.round(df[ma_keys].values, 2).T.tolist()
        for i, key, in enumerate(ma_keys):
            line.add_yaxis(
                key,
                mas[i],
                is_smooth=True,
                symbol_size=0.1,
                is_hover_animation=False,
                linestyle_opts=opts.LineStyleOpts(opacity=1),
                label_opts=opts.LabelOpts(is_show=False),
            )
    bar.overlap(line)

    if grid_index is None:
        sign = np.sign(df.close - df.open).tolist()
        bar.add_js_funcs(f"var sign = {sign}")
    return bar


def draw_indicator(
    df,
    line_keys: Union[list, None] = None,
    bar_key: Union[str, None] = None,
    line_colors: Union[list, None] = None,
    bar_color: Union[str, None] = None,
    arrow_key: Union[str, None] = None,
    arrow_colors: Union[tuple, None] = ['#28FF28', '#FF0000'],
    arrow_size=10,
    round_=2,
    category_gap='20%',
    grid_index=None,
):
    xdata = df.index.tolist()

    color_func = """
        function(params) {
            var colorList;
            if (params.data >= 0) {
                colorList = '#FF0000';
            } else {
                colorList = '#28FF28';
            }
            return colorList;
        }
    """
    bar = (
        Bar(
            init_opts=opts.InitOpts(
                theme=ThemeType.CHALK,
                bg_color='#23262F',
            )
        )
        .add_xaxis(xdata)
    )
    if bar_key is not None:
        ydata = np.round(df[bar_key].values, round_).tolist()
        if bar_color is None:
            bar_color = JsCode(color_func)

        bar.add_yaxis(
            series_name=bar_key,
            y_axis=ydata,
            category_gap=category_gap,
            xaxis_index=grid_index,
            yaxis_index=grid_index,
            label_opts=opts.LabelOpts(is_show=False),
            itemstyle_opts=opts.ItemStyleOpts(opacity=0.95, color=bar_color)
        )

    line = Line().add_xaxis(xdata)
    if line_keys is not None:
        ydatas = np.round(df[line_keys].values, round_).T.tolist()
        if line_colors is None:
            colors = [None for i in range(len(line_keys))]
        for i, key, in enumerate(line_keys):
            line.add_yaxis(
                key,
                ydatas[i],
                is_smooth=True,
                symbol_size=0.1,
                is_hover_animation=False,
                linestyle_opts=opts.LineStyleOpts(color=colors[i]),
                label_opts=opts.LabelOpts(is_show=False),
            )

    scatter = Scatter().add_xaxis(xdata)
    if arrow_key is not None:
        color_map = {1: 1, -1: 0, 0: 0}
        ydata = np.round(
            df[[arrow_key, line_keys[0]]].replace(
                {'death': -1, 'gold': 1}).values,
            round_).tolist()
        scatteritems = [
            opts.ScatterItem(
                name='',
                value=-v[0]*2+v[1],
                symbol='arrow',
                symbol_rotate=90-90*v[0],
                symbol_size=arrow_size,
                itemstyle_opts=opts.ItemStyleOpts(
                    color=arrow_colors[
                        color_map[color_map[np.nan_to_num(v[0])]]
                    ]
                ),
            ) for v in ydata
        ]

        scatter.add_yaxis(
            series_name='cross',
            y_axis=scatteritems,
            xaxis_index=grid_index,
            yaxis_index=grid_index,
            label_opts=opts.LabelOpts(is_show=False),
            itemstyle_opts=opts.ItemStyleOpts(opacity=0.95),
        )
        scatter.set_series_opts(tooltip_opts=opts.TooltipOpts(is_show=False))

    bar.overlap(line)
    bar.overlap(scatter)
    return bar

In [3]:
# init
api = sj.Shioaji(simulation=True)
api.login(
    'PAPIUSER01', '2222'
);

# contract
contract = api.Contracts.Futures.TXF.TXFR1

Response Code: 0 | Event Code: 0 | Info: host '218.32.76.102:80', hostname '218.32.76.102:80' IP 218.32.76.102:80 (host 1 of 1) (host connection attempt 1 of 1) (total connection attempt 1 of 1) | Event: Session up


In [4]:
ticks = api.ticks(
    contract=contract,
    #date='2021-11-29'
)
ticks = collate_ticks(ticks).between_time('08:45', '13:45')
df = ticks2kbar(ticks, diff=True)
df

,open,high,low,close,volume,buy_v,buy_c,buy_m,sell_v,sell_c,sell_m,v_diff,c_diff,m_diff,v_diff_cum,c_diff_cum,m_diff_cum
date,,,,,,,,,,,,,,,,,
2021-12-06 08:46:00,17619.0,17623.0,17591.0,17619.0,2722,1451,493,2.943205,1150,614,1.872964,301,-121,1.070241,301,-121,1.070241
2021-12-06 08:47:00,17617.0,17624.0,17608.0,17611.0,856,401,202,1.985149,436,233,1.871245,-35,-31,0.113904,266,-152,1.184145
2021-12-06 08:48:00,17611.0,17616.0,17605.0,17605.0,317,118,69,1.710145,196,87,2.252874,-78,-18,-0.542729,188,-170,0.641416
2021-12-06 08:49:00,17604.0,17609.0,17596.0,17605.0,397,155,82,1.890244,234,92,2.543478,-79,-10,-0.653234,109,-180,-0.011818
2021-12-06 08:50:00,17605.0,17612.0,17605.0,17611.0,250,163,78,2.089744,83,41,2.024390,80,37,0.065353,189,-143,0.053535
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-12-06 13:41:00,17687.0,17688.0,17685.0,17687.0,197,83,23,3.608696,113,45,2.511111,-30,-22,1.097585,2403,1384,-32.354830
2021-12-06 13:42:00,17686.0,17688.0,17685.0,17687.0,119,58,33,1.757576,60,39,1.538462,-2,-6,0.219114,2401,1378,-32.135716
2021-12-06 13:43:00,17687.0,17689.0,17686.0,17687.0,175,94,42,2.238095,81,47,1.723404,13,-5,0.514691,2414,1373,-31.621025


In [5]:
df = (
    df
    .pipe(cal_sma, 'close', (5, 10, 20, 60))
    .pipe(cal_sma, 'volume', (5, 10), suffix='_v')
    .pipe(cal_bbands)
    .pipe(cal_vwap, '5H')
    .pipe(cal_macd)
    .pipe(cal_kdj)
    .pipe(cal_kd_cross)
)
df

,open,high,low,close,volume,buy_v,buy_c,buy_m,sell_v,sell_c,...,bbands_percent,bbands_bw,vwap,macd_dif,macd_dem,macd_osc,k,d,j,kd_cross
date,,,,,,,,,,,,,,,,,,,,,
2021-12-06 08:46:00,17619.0,17623.0,17591.0,17619.0,2722,1451,493,2.943205,1150,614,...,NaN,NaN,17609.666667,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021-12-06 08:47:00,17617.0,17624.0,17608.0,17611.0,856,401,202,1.985149,436,233,...,NaN,NaN,17610.224893,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021-12-06 08:48:00,17611.0,17616.0,17605.0,17605.0,317,118,69,1.710145,196,87,...,NaN,NaN,17609.962430,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021-12-06 08:49:00,17604.0,17609.0,17596.0,17605.0,397,155,82,1.890244,234,92,...,NaN,NaN,17609.195092,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021-12-06 08:50:00,17605.0,17612.0,17605.0,17611.0,250,163,78,2.089744,83,41,...,NaN,NaN,17609.074270,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-12-06 13:41:00,17687.0,17688.0,17685.0,17687.0,197,83,23,3.608696,113,45,...,0.604371,0.000975,17683.823610,-0.240494,-0.975295,0.734800,77.899069,71.686549,90.324111,NaN
2021-12-06 13:42:00,17686.0,17688.0,17685.0,17687.0,119,58,33,1.757576,60,39,...,0.611991,0.000959,17683.853593,-0.123030,-0.804842,0.681812,76.175137,73.182745,82.159922,NaN
2021-12-06 13:43:00,17687.0,17689.0,17686.0,17687.0,175,94,42,2.238095,81,47,...,0.619975,0.000943,17683.909452,-0.029597,-0.649793,0.620196,75.025849,73.797113,77.483321,NaN


# one plate

In [12]:
# first
kline = draw_klines(df)
set_global_opts(
    kline,
    axis_index=[0],
    grid_index=0,
    show_xaxis=True,
    title='2330',
    legend_opts={'is_show': True, 'pos_top': '0%'},
    toolbox_position=('0.5%', '7%'),
    tooltip_position=('85%', '7%'),
    range_=(60, 100),
)

grid = (
    Grid(
        init_opts=opts.InitOpts(
            width='1000px',
            height='800px',
            theme=ThemeType.CHALK,
            bg_color="#23262F",
        )
    )
    .add(kline, grid_opts=opts.GridOpts(pos_left="9%", pos_right="15%", height="85%"))

)
grid.render_notebook()

# two plate

In [6]:
# first
kline = draw_klines(df)
set_global_opts(
    kline,
    axis_index=[0, 1],
    grid_index=0,
    show_xaxis=False,
    title='2330',
    legend_opts={'is_show': True, 'pos_top': '0%'},
    toolbox_position=('0.5%', '7%'),
    tooltip_position=('85%', '7%'),
    range_=(60, 100),
)
bar = draw_volume(df, category_gap='1%')
set_global_opts(
    bar,
    y_split_number=2,
    legend_opts={'is_show': True, 'pos_top': '5%'},
)

grid = (
    Grid(
        init_opts=opts.InitOpts(
            width='1000px',
            height='800px',
            theme=ThemeType.CHALK,
            bg_color="#23262F",
        )
    )
    .add(kline, grid_opts=opts.GridOpts(pos_left="9%", pos_right="15%", height="60%"))
    .add(bar, grid_opts=opts.GridOpts(pos_left="9%", pos_right="15%", pos_top="70%", height="16%"))

)
sign = np.sign(df.close - df.open).tolist()
grid.add_js_funcs(f"var sign = {sign}")
grid.render_notebook()

# three plate

In [14]:
# first
kline = draw_klines(df)
set_global_opts(
    kline,
    y_split_number=5,
    axis_index=[0, 1, 2],
    show_xaxis=False,
    title='TXFR1',
    legend_opts={'is_show': True, 'pos_top': '0%', 'pos_left': '9%'},
    toolbox_position=('0.5%', '7%'),
    tooltip_position=('86%', '7%'),
    always_show_tooltip_content=True,
    range_=(60, 100),
    datazoom_position='98%',
)

# second
bar = draw_volume(df, category_gap='1%')
set_global_opts(
    bar,
    show_xaxis=False,
    y_split_number=2,
    legend_opts={'is_show': True, 'pos_top': '3%', 'pos_left': '9%'},
)

# third
indicator1 = draw_indicator(df, ['k', 'd', 'j'], arrow_key='kd_cross')
set_global_opts(
    indicator1,
    y_split_number=2,
    show_xaxis=True,
    legend_opts={'is_show': True, 'pos_top': '3%', 'pos_left': '35%'},
)

grid = (
    Grid(
        init_opts=opts.InitOpts(
            width='1100px',
            height='800px',
            theme=ThemeType.CHALK,
            bg_color="#23262F",
        )
    )
    .add(kline, grid_opts=opts.GridOpts(pos_left="9%", pos_right="15%", pos_top="6%", height="50%"))
    .add(bar, grid_opts=opts.GridOpts(pos_left="9%", pos_right="15%", pos_top="60%", height="15%"))
    .add(indicator1, grid_opts=opts.GridOpts(pos_left="9%", pos_right="15%", pos_top="75%", height="20%"))
)
sign = np.sign(df.close - df.open).tolist()
grid.add_js_funcs(f"var sign = {sign}")
grid.page_title = 'test'
grid.render_notebook()

# four plate

In [15]:
# first
kline = draw_klines(df)
set_global_opts(
    kline,
    y_split_number=5,
    axis_index=[0, 1, 2, 3],
    show_xaxis=False,
    title='TXFR1',
    legend_opts={'is_show': True, 'pos_top': '0%', 'pos_left': '9%'},
    toolbox_position=('0.5%', '7%'),
    tooltip_position=('86%', '7%'),
    always_show_tooltip_content=True,
    range_=(60, 100),
    datazoom_position='98%',
)

# second
bar = draw_volume(df, category_gap='1%')
set_global_opts(
    bar,
    show_xaxis=False,
    y_split_number=2,
    legend_opts={'is_show': True, 'pos_top': '3%', 'pos_left': '9%'},
)

# third
indicator1 = draw_indicator(df, ['k', 'd', 'j'], arrow_key='kd_cross')
set_global_opts(
    indicator1,
    y_split_number=2,
    show_xaxis=False,
    legend_opts={'is_show': True, 'pos_top': '3%', 'pos_left': '35%'},
)
indicator2 = draw_indicator(
    df, ['macd_dif', 'macd_dem'], 'macd_osc', category_gap=0)
set_global_opts(
    indicator2,
    y_split_number=2,
    legend_opts={'is_show': True, 'pos_top': '3%', 'pos_left': '60%'},
)

grid = (
    Grid(
        init_opts=opts.InitOpts(
            width='1100px',
            height='800px',
            theme=ThemeType.CHALK,
            bg_color="#23262F",
        )
    )
    .add(kline, grid_opts=opts.GridOpts(pos_left="9%", pos_right="15%", pos_top="6%", height="35%"))
    .add(bar, grid_opts=opts.GridOpts(pos_left="9%", pos_right="15%", pos_top="43%", height="12%"))
    .add(indicator1, grid_opts=opts.GridOpts(pos_left="9%", pos_right="15%", pos_top="57.5%", height="18%"))
    .add(indicator2, grid_opts=opts.GridOpts(pos_left="9%", pos_right="15%", pos_top="77%", height="18%"))
)
sign = np.sign(df.close - df.open).tolist()
grid.add_js_funcs(f"var sign = {sign}")
grid.page_title = 'test'
grid.render_notebook()

# five plate

In [8]:
# first
kline = draw_klines(df)
set_global_opts(
    kline,
    y_split_number=5,
    axis_index=[0, 1, 2, 3, 4],
    show_xaxis=False,
    title='TXFR1',
    legend_opts={'is_show': True, 'pos_top': '0%', 'pos_left': '9%'},
    toolbox_position=('0.5%', '7%'),
    tooltip_position=('86%', '7%'),
    always_show_tooltip_content=True,
    range_=(0, 100),
    datazoom_position='98%',
)

# second
bar = draw_volume(df, category_gap='1%')
set_global_opts(
    bar,
    show_xaxis=False,
    y_split_number=2,
    legend_opts={'is_show': True, 'pos_top': '3%', 'pos_left': '9%'},
)

# third
indicator1 = draw_indicator(
    df, ['c_diff'], 'c_diff_cum', category_gap='0%')
set_global_opts(
    indicator1,
    y_split_number=2,
    show_xaxis=False,
    legend_opts={'is_show': True, 'pos_top': '3%', 'pos_left': '35%'},
)
# 4th
indicator2 = draw_indicator(
    df, ['v_diff'], 'v_diff_cum', category_gap='0%')
set_global_opts(
    indicator2,
    y_split_number=4,
    show_xaxis=False,
    legend_opts={'is_show': True, 'pos_top': '3%', 'pos_left': '60%'},
)
# 5th
indicator3 = draw_indicator(
    df, ['m_diff'], 'm_diff_cum', category_gap='0%')
set_global_opts(
    indicator3,
    y_split_number=4,
    show_xaxis=True,
    legend_opts={'is_show': True, 'pos_top': '3%', 'pos_left': '80%'},
)

grid = (
    Grid(
        init_opts=opts.InitOpts(
            width='1100px',
            height='800px',
            theme=ThemeType.CHALK,
            bg_color="#23262F",
        )
    )
    .add(kline, grid_opts=opts.GridOpts(pos_left="9%", pos_right="15%", pos_top="6%", height="25%"))
    .add(bar, grid_opts=opts.GridOpts(pos_left="9%", pos_right="15%", pos_top="33%", height="12%"))
    .add(indicator1, grid_opts=opts.GridOpts(pos_left="9%", pos_right="15%", pos_top="45%", height="16.5%"))
    .add(indicator2, grid_opts=opts.GridOpts(pos_left="9%", pos_right="15%", pos_top="64%", height="16.5%"))
    .add(indicator3, grid_opts=opts.GridOpts(pos_left="9%", pos_right="15%", pos_top="83%", height="16.5%"))
)
sign = np.sign(df.close - df.open).tolist()
grid.add_js_funcs(f"var sign = {sign}")
grid.page_title = 'test'
grid.render_notebook()
